In [1]:
import pandas as pd
import gzip

In [2]:
LOCAL = 'Lausanne'

In [3]:
# Specify the file paths
activity_file = 'data_original/vaud_activities.csv.gz'
population_file = 'data_original/vaud_population.csv.gz'
trip_file = 'data_original/vaud_trips.csv.gz'

# Read the gzipped CSV files
def read_gzipped_csv(file_path):
    with gzip.open(file_path, 'rt') as file:
        df = pd.read_csv(file)
    return df

# Read the dataframes
activity_vaud = read_gzipped_csv(activity_file)
population_vaud = read_gzipped_csv(population_file)[['id', 'age', 'home_x', 'home_y', 'local']].drop_duplicates()
trip_vaud = read_gzipped_csv(trip_file)[['Unnamed: 0', 'id', 'mode', 'dep_time','trav_time','start_link','end_link']].drop_duplicates()

print("File opened and read \n")

File opened and read 



In [4]:
population_local = population_vaud[population_vaud['local'] == LOCAL]
population_local_ids = population_local['id'].unique()

population_local.head(1)


Here's the count of facilities by types in Lausanne :              type  facility
0       education       220
1            home     70975
2         leisure      6277
3           other     10178
4  pt interaction         0
5            shop      3177
6            work     13173

Average durations of trips in Lausanne : round(10.86828355)m


C:\Users\helli\AppData\Local\Temp\ipykernel_16192\2216327453.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_local['trav_time'] = pd.to_timedelta(trip_local['trav_time'])


,id,age,home_x,home_y,local
34200,6336880,30,2539957.0,1152783.0,Lausanne


In [5]:
activity_local = activity_vaud[activity_vaud['id'].isin(population_local_ids)]
count_act_by_types = activity_local.groupby('type')['facility'].nunique().reset_index()
print(f"\nHere's the count of facilities by types in {LOCAL} : {count_act_by_types}")
activity_local_filtered = activity_local[~activity_local['type'].isin(['other', 'pt interaction'])]


Here's the count of facilities by types in Lausanne :              type  facility
0       education       220
1            home     70975
2         leisure      6277
3           other     10178
4  pt interaction         0
5            shop      3177
6            work     13173


In [7]:
trip_local = trip_vaud[trip_vaud['id'].isin(population_local_ids)]
trip_local['trav_time'] = pd.to_timedelta(trip_local['trav_time'])
average_duration = trip_local['trav_time'].mean().total_seconds() / 60  # convertir en minutes TO CHECK
print(f"\nAverage durations of trips in {LOCAL} : round({average_duration})m")

trip_local.head()


Average durations of trips in Lausanne : round(10.86828355)m


C:\Users\helli\AppData\Local\Temp\ipykernel_16192\388021354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_local['trav_time'] = pd.to_timedelta(trip_local['trav_time'])


,Unnamed: 0,id,mode,dep_time,trav_time,start_link,end_link
147102,25983202,6336880,car,07:06:35,0 days 00:10:06,290992,112618
147103,25983203,6336880,car,16:51:35,0 days 00:00:00,112618,112618
147104,25983204,6336880,car,19:06:35,0 days 00:14:36,112618,771858
147105,25983205,6336880,car,21:36:35,0 days 00:14:38,771858,290992
147106,25983206,6336882,bike,07:17:55,0 days 00:07:17,562016,820641


In [24]:
work_facilities_df = activity_local_filtered[activity_local_filtered['type'] == 'work'][['id', 'facility', 'x', 'y']].drop_duplicates() # verifier via les homes
population_local_merged = population_local.merge(work_facilities_df, on='id', how='left')
population_local_merged.rename(columns={'facility': 'work_facility', 'x': 'work_facility_x', 'y': 'work_facility_y'}, inplace=True)

population_local_merged.head()

,id,age,home_x,home_y,local,work_facility,work_facility_x,work_facility_y,work_facility,work_facility_x,...,work_facility_y,work_facility,work_facility_x,work_facility_y,work_facility,work_facility_x,work_facility_y,work_facility,work_facility_x,work_facility_y
0,6336880,30,2539957.0,1152783.0,Lausanne,95509,2538446.0,1152412.0,95509,2538446.0,...,1152412.0,95509,2538446.0,1152412.0,95509,2538446.0,1152412.0,95509,2538446.0,1152412.0
1,6336884,48,2539610.0,1152921.0,Lausanne,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6336892,26,2539600.0,1152942.0,Lausanne,17888,2533582.0,1154867.0,17888,2533582.0,...,1154867.0,17888,2533582.0,1154867.0,17888,2533582.0,1154867.0,17888,2533582.0,1154867.0
3,6336928,85,2539515.0,1153650.0,Lausanne,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6336950,39,2539738.0,1152690.0,Lausanne,90780,2537984.0,1151737.0,90780,2537984.0,...,1151737.0,90780,2537984.0,1151737.0,90780,2537984.0,1151737.0,90780,2537984.0,1151737.0


In [21]:
# DO NOT RUN BELOW : JOIN TO GET TRIP DISTANCE IS TOO HEAVY
# je pense que si je me penche moi meme dessus, je peux le resoudre

In [7]:
activity_local.set_index('link', inplace=True)

trip_local = trip_local.join(activity_local[['x', 'y']], on='start_link', how='left', rsuffix='_start')

MemoryError: Unable to allocate 6.48 GiB for an array with shape (2, 434616670) and data type object

In [6]:
trip_local = trip_local.merge(activity_local[['link', 'x', 'y']], left_on='start_link', right_on='link', how='left')
trip_local.rename(columns={'x': 'start_x', 'y': 'start_y'}, inplace=True)
trip_local.drop(columns=['link'], inplace=True)

trip_local.head()

In [ ]:
trip_local = trip_local.merge(activity_local[['link', 'x', 'y']], left_on='end_link', right_on='link', how='left')
trip_local.rename(columns={'x': 'end_x', 'y': 'end_y'}, inplace=True)
trip_local.drop(columns=['link'], inplace=True)

trip_local.head()

In [ ]:
trip_local['distance'] = np.sqrt((trip_local['start_x'] - trip_local['end_x']) ** 2 +
                                (trip_local['start_y'] - trip_local['end_y']) ** 2)
trip_local['average_speed'] = (trip_local['distance'] / 1000) / (trip_local['trav_time'].dt.total_seconds() / 3600)

average_distance = trip_local['distance'].mean() / 1000  # convert to km
overall_average_speed = trip_local['average_speed'].mean()

print("Average Distance: {:.2f} km".format(average_distance))
print("Overall Average Speed: {:.2f} km/h".format(overall_average_speed))